# 10. Analyzing the Meaning of Sentences

## 1   Natural Language Understanding

### 1.1   Querying a Database

![](figures/table_1.1.png)

In [3]:
import nltk

In [4]:
nltk.data.show_cfg('grammars/book_grammars/sql0.fcfg')

% start S
S[SEM=(?np + WHERE + ?vp)] -> NP[SEM=?np] VP[SEM=?vp]
VP[SEM=(?v + ?pp)] -> IV[SEM=?v] PP[SEM=?pp]
VP[SEM=(?v + ?ap)] -> IV[SEM=?v] AP[SEM=?ap]
NP[SEM=(?det + ?n)] -> Det[SEM=?det] N[SEM=?n]
PP[SEM=(?p + ?np)] -> P[SEM=?p] NP[SEM=?np]
AP[SEM=?pp] -> A[SEM=?a] PP[SEM=?pp]
NP[SEM='Country="greece"'] -> 'Greece'
NP[SEM='Country="china"'] -> 'China'
Det[SEM='SELECT'] -> 'Which' | 'What'
N[SEM='City FROM city_table'] -> 'cities'
IV[SEM=''] -> 'are'
A[SEM=''] -> 'located'
P[SEM=''] -> 'in'


In [71]:
from nltk import load_parser
cp = load_parser('grammars/book_grammars/sql0.fcfg')
query = 'What cities are located in Greece'
trees = list(cp.parse(query.split()))
answer = trees[0].label()['SEM']
answer = [s for s in answer if s]
q = ' '.join(answer)
print(q)

SELECT City FROM city_table WHERE Country="greece"


In [66]:
trees

[Tree(S[SEM=(SELECT, City FROM city_table, WHERE, , , Country="china")], [Tree(NP[SEM=(SELECT, City FROM city_table)], [Tree(Det[SEM='SELECT'], ['What']), Tree(N[SEM='City FROM city_table'], ['cities'])]), Tree(VP[SEM=(, , Country="china")], [Tree(IV[SEM=''], ['are']), Tree(AP[SEM=(, Country="china")], [Tree(A[SEM=''], ['located']), Tree(PP[SEM=(, Country="china")], [Tree(P[SEM=''], ['in']), Tree(NP[SEM='Country="china"'], ['China'])])])])])]

In [6]:
from nltk.sem import chat80
rows = chat80.sql_query('corpora/city_database/city.db', q)
for r in rows: print(r[0], end=" ") 

canton chungking dairen harbin kowloon mukden peking shanghai sian tientsin 

In [ ]:
지금까지의 방법은 미리 SQL과 영어 단어간의 관계를 정해야 사용할 수 있었다.

### 1.2   Natural Language, Semantics and Logic

In [ ]:
Which country is Athens in?
문장을 SQL로 번역해서 컴퓨터가 이해하고 실행시킬 수 있도록 했음, 번역이 정확한지 의문
파악한 문장의 의미가 특정 상황에서 사실인지 아닌

Margrietje houdt van Brunoke.
이것의 의미를 이해한다면 아래 그림의 상황에서 이 문장이 사실이라는 것을 알 수 있다.

문장이 특정상황에서 true or false
noun phrase와 proper nouns이 사물을 참조

![](figures/figure_1.1.png)

In [ ]:
(5) consistent, (6), (7) inconsistent

![](figures/figure_1.2.png)

In [ ]:
모델 표현은 집합 이론을 주로 이용
k: Klaus, s: Stefan, e: Evi

## 2   Propositional Logic

In [ ]:
boolean operators: 접속사의 다른말
propositional symbols: propositional logic의 기본 표현, P, Q, R ..

In [10]:
nltk.boolean_ops()

negation       	-
conjunction    	&
disjunction    	|
implication    	->
equivalence    	<->


![](figures/table_2.1.png)

In [12]:
read_expr = nltk.sem.Expression.fromstring
read_expr('-(P & Q)')

<NegatedExpression -(P & Q)>

In [13]:
read_expr('P & Q')

<AndExpression (P & Q)>

In [14]:
read_expr('P | (R -> Q)')

<OrExpression (P | (R -> Q))>

In [15]:
read_expr('P <-> -- P')

<IffExpression (P <-> --P)>

In [83]:
lp = nltk.sem.Expression.fromstring
SnF = read_expr('SnF')
NotFnS = read_expr('-FnS')
R = read_expr('SnF -> -FnS')
prover = nltk.Prover9()
prover.prove(NotFnS, [SnF, R])

True

In [84]:
lp = nltk.sem.Expression.fromstring
SnF = read_expr('SnF')
NotFnS = read_expr('-FnS')
R = read_expr('SnF -> FnS')
prover = nltk.Prover9()
prover.prove(NotFnS, [SnF, R])

False

SnF -> -FnS == -SnF | -FnS, 
"|" == or


φ | ψ is true in a situation s 이면,
φ is true in s 이거나 ψ is true in s. 

이제 SnF와 -SnF | -FnS 가 true in situation s라고 할때
SnF is true이면 -SnF 은 true가 될 수 없다.

기본적인 가정은 어떠한 문장이 동시 true이면서 false일 수 없다는 것.

결과적으로 -FnS must be true.

In [86]:
val = nltk.Valuation([('P', True), ('Q', True), ('R', False)])
val['P']

True

In [87]:
val['Q']

True

In [88]:
val['R']

False

In [89]:
dom = set()
g = nltk.Assignment(dom)
m = nltk.Model(dom, val)
print(m.evaluate('(P & Q)', g))
print(m.evaluate('-(P & Q)', g))
print(m.evaluate('(P & R)', g))
print(m.evaluate('(P | R)', g))

True
False
False
True


## 3   First-Order Logic

### 3.1   Syntax

In [95]:
read_expr = nltk.sem.Expression.fromstring
expr = read_expr('walk(angus)', type_check=True)
expr.argument

<ConstantExpression angus>

In [23]:
expr.argument.type

e

In [24]:
expr.function

<ConstantExpression walk>

In [25]:
expr.function.type

<e,?>

In [94]:
read_expr = nltk.sem.Expression.fromstring
expr = read_expr('see(angus, bertie)', type_check=True)
expr.function.type

<e,?>

In [96]:
sig = {'walk': '<e, t>'}
expr = read_expr('walk(angus)', signature=sig)
expr.function

<ConstantExpression walk>

In [97]:
expr.function.type

e

In [ ]:
He 대명사를 해석하는 방법은 로컬 맥락에서 관련된 개인을 가리키는 것

In [30]:
read_expr = nltk.sem.Expression.fromstring
read_expr('dog(cyril)').free()

set()

In [31]:
read_expr('dog(x)').free()

{Variable('x')}

In [32]:
read_expr('own(angus, cyril)').free()

set()

In [33]:
read_expr('exists x.dog(x)').free()

set()

In [34]:
read_expr('((some x. walk(x)) -> sing(x))').free()

{Variable('x')}

In [35]:
read_expr('exists x.own(y, x)').free()

{Variable('y')}

### 3.2   First Order Theorem Proving

In [92]:
NotFnS = read_expr('-north_of(f, s)') # proof goal
SnF = read_expr('north_of(s, f)') # assumption
R = read_expr('all x. all y. (north_of(x, y) -> -north_of(y, x))') # assumption
prover = nltk.Prover9()
prover.prove(NotFnS, [SnF, R])

True

In [76]:
FnS = read_expr('north_of(f, s)')
prover.prove(FnS, [SnF, R])

False

### 3.3   Summarizing the Language of First Order Logic

![](figures/table_3.1.png)

### 3.4   Truth in Model

In [39]:
dom = {'b', 'o', 'c'}
v = """
    bertie => b
    olive => o
    cyril => c
    boy => {b}
    girl => {o}
    dog => {c}
    walk => {o, c}
    see => {(b, o), (c, b), (o, c)}
    """
val = nltk.Valuation.fromstring(v)
print(val)

{'bertie': 'b',
 'boy': {('b',)},
 'cyril': 'c',
 'dog': {('c',)},
 'girl': {('o',)},
 'olive': 'o',
 'see': {('b', 'o'), ('c', 'b'), ('o', 'c')},
 'walk': {('c',), ('o',)}}


In [40]:
('o', 'c') in val['see']

True

In [41]:
('b',) in val['boy']

True

### 3.5   Individual Variables and Assignments

In [42]:
g = nltk.Assignment(dom, [('x', 'o'), ('y', 'c')])
g

{'x': 'o', 'y': 'c'}

In [43]:
print(g)

g[c/y][o/x]


In [44]:
m = nltk.Model(dom, val)
m.evaluate('see(olive, y)', g)

True

In [45]:
g['y']

'c'

In [46]:
m.evaluate('see(y, x)', g)

False

In [47]:
g.purge()
g

{}

In [48]:
m.evaluate('see(olive, y)', g)

'Undefined'

In [49]:
m.evaluate('see(bertie, olive) & boy(bertie) & -walk(bertie)', g)

True

### 3.6   Quantification

In [50]:
m.evaluate('exists x.(girl(x) & walk(x))', g)

True

In [51]:
m.evaluate('girl(x) & walk(x)', g.add('x', 'o'))

True

In [52]:
fmla1 = read_expr('girl(x) | boy(x)')
m.satisfiers(fmla1, 'x', g)

{'b', 'o'}

In [53]:
fmla2 = read_expr('girl(x) -> walk(x)')
m.satisfiers(fmla2, 'x', g)

{'b', 'c', 'o'}

In [54]:
fmla3 = read_expr('walk(x) -> girl(x)')
m.satisfiers(fmla3, 'x', g)

{'b', 'o'}

In [55]:
m.evaluate('all x.(girl(x) -> walk(x))', g)

True

### 3.7   Quantifier Scope Ambiguity

In [56]:
v2 = """
    bruce => b
    elspeth => e
    julia => j
    matthew => m
    person => {b, e, j, m}
    admire => {(j, b), (b, b), (m, e), (e, m)}
    """
val2 = nltk.Valuation.fromstring(v2)

In [57]:
dom2 = val2.domain
m2 = nltk.Model(dom2, val2)
g2 = nltk.Assignment(dom2)
fmla4 = read_expr('(person(x) -> exists y.(person(y) & admire(x, y)))')
m2.satisfiers(fmla4, 'x', g2)

{'b', 'e', 'j', 'm'}

In [58]:
fmla5 = read_expr('(person(y) & all x.(person(x) -> admire(x, y)))')
m2.satisfiers(fmla5, 'y', g2)

set()

In [59]:
fmla6 = read_expr('(person(y) & all x.((x = bruce | x = julia) -> admire(x, y)))')
m2.satisfiers(fmla6, 'y', g2)

{'b'}

### 3.8   Model Building

In [77]:
a3 = read_expr('exists x.(man(x) & walks(x))')
c1 = read_expr('mortal(socrates)')
c2 = read_expr('-mortal(socrates)')
mb = nltk.Mace(5)
print(mb.build_model(None, [a3, c1]))
print(mb.build_model(None, [a3, c2]))
print(mb.build_model(None, [c1, c2]))

True
True
False


In [78]:
a4 = read_expr('exists y. (woman(y) & all x. (man(x) -> love(x,y)))')
a5 = read_expr('man(adam)')
a6 = read_expr('woman(eve)')
g = read_expr('love(adam,eve)')
mc = nltk.MaceCommand(g, assumptions=[a4, a5, a6])
mc.build_model()

True

In [79]:
print(mc.valuation)

{'C1': 'b',
 'adam': 'a',
 'eve': 'a',
 'love': {('a', 'b')},
 'man': {('a',)},
 'woman': {('b',), ('a',)}}


In [80]:
a7 = read_expr('all x. (man(x) -> -woman(x))')
g = read_expr('love(adam,eve)')
mc = nltk.MaceCommand(g, assumptions=[a4, a5, a6, a7])
mc.build_model()

True

In [81]:
print(mc.valuation)

{'C1': 'c',
 'adam': 'a',
 'eve': 'b',
 'love': {('a', 'c')},
 'man': {('a',)},
 'woman': {('c',), ('b',)}}
